# Import MOD09GA, MODOCGA, MOD11A1 and MYD11A1 Data

This is the priliminary notebook of a serie of three:
- 00_ImportData: imports products from Terra as well as Aqua and merges it into one dataframe
- 01_QualitzControl: invalid entries of the data are replaced by NA 
- 02_MissingData_complete: the NA entries are filled depending on the chosen method can be analysed in 
- 03_Analysis: The input variables are analysed, the remaining NA values are deleted, further information from the sites is added and machine Learning is used to analyse the data and train a model to allow conclusions about soil moisture based on remotely sensing data

#### This notebook contains:
- the import of data in different folder structures 
- the creation of a dataframe containing all relevant information (data/df_lst_and_refl.csv)

In [1]:
# enables the use of R code in a cell after writing "%%R making it an cell for R code 
%load_ext rpy2.ipython   

Most of the R libraries will only be added in later noteboods, here we only use tidzverse.
rbeni and ingestr is not available in CRAN and must be loaded from github repositories:
- https://github.com/stineb/rbeni
- https://github.com/computationales/ingestr
These two packets are actually not used in this code anymore, since everythin gis already downloaded. However should a new download be necessary they can prove to be useful. Rbeni will be used for plots in the analysis part.

In [2]:
%%capture
%%R 
# path_rbeni = "/home/zhud/wsl/MasterThesis/rsvi/data/rbeni-master/"
# path_ingestr = "/home/zhud/wsl/MasterThesis/rsvi/data/ingestr-master/"

In [3]:
%%capture
%%R 
# set working directory
setwd("/home/zhud/wsl/MasterThesis/rsvi")

# import R libaries
library(tidyverse)              

Failed to create bus connection: No such file or directory


In [4]:
# import libraries in python
import pandas as pd
import glob
import rpy2.robjects as ro                            # allows the use of R code 
from rpy2.robjects.packages import importr            # making importing R packages possible, i.e.: utils = importr('utils')
base = importr('base')
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import os.path
import csv
from pdb import set_trace
import add_fluxnet_modis
import numpy as np
import pyreadr
from functools import partial
from rpy2.ipython import html
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

### Import data 

- MOD09GA (band 1 to 7) 
- MODOCGA (band 8 to 16)

In [5]:
%%R
raw_dir = './data/FLUXNET_MODOCGA_MOD09GA1km_2000_2018'

In [6]:
%%R -o data_stacked 
csv_files = list.files(path = raw_dir, pattern = "csv$", full.names = TRUE)
data_stacked <- map_dfr(csv_files, read_csv)

Rows: 6826 Columns: 48
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): system_index, igbp_flx, sites_id, x_geo, Var44, Var45
dbl (42): YY, MM, DD, QC_500m, QC_b16_1km, QC_b8_15_1km, Range, SensorAzimut...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 6826 Columns: 48
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): system_index, igbp_flx, sites_id, x_geo, Var44, Var45
dbl (42): YY, MM, DD, QC_500m, QC_b16_1km, QC_b8_15_1km, Range, SensorAzimut...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 6826 Columns: 48
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): system_index, igbp_flx, sites_id, x

In [7]:
refl_df = data_stacked[['YY', 'sites_id', 'site_num', 'state_1km',  'MM', 'DD', 'QC_500m', 'QC_b8_15_1km', 'QC_b16_1km', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'sur_refl_b16']]

In [8]:
print(refl_df.dtypes)      

YY              float64
sites_id         object
site_num        float64
state_1km       float64
MM              float64
DD              float64
QC_500m         float64
QC_b8_15_1km    float64
QC_b16_1km      float64
sur_refl_b01    float64
sur_refl_b02    float64
sur_refl_b03    float64
sur_refl_b04    float64
sur_refl_b05    float64
sur_refl_b06    float64
sur_refl_b07    float64
sur_refl_b16    float64
dtype: object


In [9]:
refl_df = refl_df.astype({"YY": 'int',
                "MM": 'int',
                "DD": 'int'})

In [10]:
refl_df['id'] = refl_df["YY"].astype(str) + '_' + refl_df["MM"].astype(str).str.zfill(2) + '_' + refl_df["DD"].astype(str).str.zfill(2)
refl_df = refl_df.rename(columns={'sites_id':'sites'})
refl_df = refl_df.rename(columns={'QC_b8_15_1km':'QC_b8_15_1km_paula'})

In [11]:
refl_df

,YY,sites,site_num,state_1km,MM,DD,QC_500m,QC_b8_15_1km_paula,QC_b16_1km,sur_refl_b01,sur_refl_b02,sur_refl_b03,sur_refl_b04,sur_refl_b05,sur_refl_b06,sur_refl_b07,sur_refl_b16,id
1,2000,AR-SLu,1.0,1033.0,2,24,1.073742e+09,3.722305e+09,208.0,4777.000000,4977.000000,4746.000000,4831.000000,4672.000000,2809.000000,1678.000000,-100.0,2000_02_24
2,2000,AR-SLu,1.0,72.0,2,25,1.073742e+09,3.707765e+09,208.0,798.750001,2284.250001,428.000001,759.000001,2677.750001,2368.250000,1441.000000,-100.0,2000_02_25
3,2000,AR-SLu,1.0,1033.0,2,26,1.073742e+09,3.722305e+09,208.0,6264.750011,6624.250010,5941.750010,6277.000011,5583.000008,4895.500006,1710.000000,-100.0,2000_02_26
4,2000,AR-SLu,1.0,72.0,2,27,1.075577e+09,3.489661e+09,208.0,658.750000,2321.250000,367.500000,703.250000,3034.250000,2170.750000,1626.250000,-100.0,2000_02_27
5,2000,AR-SLu,1.0,72.0,2,28,1.073742e+09,3.489661e+09,208.0,522.000000,1758.250001,277.750000,525.250000,2130.250001,1775.750000,995.750000,-100.0,2000_02_28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447108,2018,ZM-Mon,212.0,8328.0,12,27,1.073742e+09,3.503292e+09,208.0,719.000000,2465.250001,388.000000,647.750000,2773.250001,2228.000001,1404.750000,-100.0,2018_12_27
1447109,2018,ZM-Mon,212.0,1033.0,12,28,1.073742e+09,3.722305e+09,208.0,8137.999997,8374.999998,8046.499997,8047.249997,8202.250003,5512.749999,3411.500003,-100.0,2018_12_28
1447110,2018,ZM-Mon,212.0,1033.0,12,29,1.073742e+09,3.722305e+09,208.0,7362.249999,7297.499999,7450.749999,7412.499999,5657.999996,2049.250000,1126.249999,-100.0,2018_12_29
1447111,2018,ZM-Mon,212.0,1033.0,12,30,1.073742e+09,3.722305e+09,208.0,3308.000000,4458.000000,3377.000000,3355.000000,4866.000000,3498.000000,2288.000000,-100.0,2018_12_30


### Import data 
- MODOCGA (band 8 to 15)
- MOD11A1 (modis_lst_terra)
- MYD11A1 (modis_lst_aqua)

In [12]:
%%R -o rsd 
# reads in the .rda file and exports it to python
rsd <- readRDS('./data/remote_sensing_data.rda')



In [13]:
%%R -o names_data
names_data <- names(rsd)
names_data

 [1] "modis_lst_terra" "modis_lst_aqua"  "modis_refl_1"    "modis_refl_2"   
 [5] "modis_refl_3"    "modis_refl_4"    "modis_refl_5"    "modis_refl_6"   
 [9] "modis_refl_7"    "modis_refl_8"    "modis_refl_9"    "modis_refl_10"  
[13] "modis_refl_11"   "modis_refl_12"   "modis_refl_13"   "modis_refl_14"  
[17] "modis_refl_15"  


### Read in all the data for each source in names_data

In [14]:
def flatten_df(data, product_number):
    '''
    data            nested data with product at the first level
                    at the second level there is the sites 
                    for each product a dataframe (df) is available
    product_number  give a number from which the df should be returned from
    return df       dataframe for the requested product_number and all sites.
                    The name of the sites are saved in a column called 'sites'    
    '''
    df = pd.DataFrame()
    sites = data[product_number][0]
    data_vector = data[product_number][1]
    for j in range(len(sites)):
        site = sites[j]
        # convert data_vector[j] into an pd df and append it and add a column with the site
        with localconverter(ro.default_converter + pandas2ri.converter):
            data = ro.conversion.rpy2py(data_vector[j])
        data['sites'] = site
        df = df.append(data)
    return df

In [15]:
# read in all products
modis_lst_terra = flatten_df(rsd, 0) # MOD11A1
modis_lst_aqua = flatten_df(rsd, 1)  # MYD11A1
modis_refl_8 = flatten_df(rsd, 9)    # MODOCGA
modis_refl_9 = flatten_df(rsd, 10)
modis_refl_10 = flatten_df(rsd, 11)
modis_refl_11 = flatten_df(rsd, 12)
modis_refl_12 = flatten_df(rsd, 13)
modis_refl_13 = flatten_df(rsd, 14)
modis_refl_14 = flatten_df(rsd, 15)
modis_refl_15 = flatten_df(rsd, 16)

In [16]:
# rename columns with same name
modis_lst_terra_newname = modis_lst_terra.rename(columns={'LST_Day_1km':'LST_Day_1km_terra','QC_Day':'QC_Day_terra'})
modis_lst_aqua_newname = modis_lst_aqua.rename(columns={'LST_Day_1km':'LST_Day_1km_aqua','QC_Day':'QC_Day_aqua'})

In [17]:
# combine lst_terra (MOD11A1) and lst_aqua (MYD11A1) in one ddf
ddf = modis_lst_terra_newname.merge(modis_lst_aqua_newname[['date', 'year_dec', 'sites',	'LST_Day_1km_aqua', 'QC_Day_aqua']], on=["date", "sites"])

In [18]:
%%R -i ddf
summary(ddf)   
# rows 265177
# time 1991 - 2015 (aqua and terra were only in the early 2000s launched, however there is little old data and it will be sorted out)
# NA lst_terra  162614
#    lst_aqua   176474

      date         year_dec_x        id            LST_Day_1km_terra
 Min.   : 7670   Min.   :1991   Length:265177      Min.   :234.2    
 1st Qu.:12464   1st Qu.:2004   Class :character   1st Qu.:284.3    
 Median :13919   Median :2008   Mode  :character   Median :293.6    
 Mean   :13696   Mean   :2007                      Mean   :292.5    
 3rd Qu.:15151   3rd Qu.:2011                      3rd Qu.:301.7    
 Max.   :16435   Max.   :2015                      Max.   :330.6    
                                                   NA's   :162614   
  QC_Day_terra       sites             year_dec_y   LST_Day_1km_aqua
 Min.   :  0.00   Length:265177      Min.   :1991   Min.   :233.9   
 1st Qu.:  2.00   Class :character   1st Qu.:2004   1st Qu.:286.1   
 Median :  2.00   Mode  :character   Median :2008   Median :295.0   
 Mean   : 12.91                      Mean   :2007   Mean   :294.9   
 3rd Qu.:  2.00                      3rd Qu.:2011   3rd Qu.:304.3   
 Max.   :145.00                   

In [19]:
# combine MODOCGA (band 8 to 15) with ddf containing already the lst data 177309 rows
# !!!!!!!!!!!!!!! merging on year_dec loses about 1/3 of the data! 
# it is calculated differently and when merging there is a slight numeric difference. 
ddf = ddf.merge(modis_refl_8[['date', 'sites', 'sur_refl_b08']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_9[['date', 'sites', 'sur_refl_b09']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_10[['date', 'sites', 'sur_refl_b10']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_11[['date', 'sites', 'sur_refl_b11']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_12[['date', 'sites', 'sur_refl_b12']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_13[['date', 'sites', 'sur_refl_b13']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_14[['date', 'sites', 'sur_refl_b14']], on=["date", "sites"])
ddf = ddf.merge(modis_refl_15[['date', 'sites', 'sur_refl_b15',	'QC_b8_15_1km']], on=["date", "sites"])
ddf

,date,year_dec_x,id,LST_Day_1km_terra,QC_Day_terra,sites,year_dec_y,LST_Day_1km_aqua,QC_Day_aqua,sur_refl_b08,sur_refl_b09,sur_refl_b10,sur_refl_b11,sur_refl_b12,sur_refl_b13,sur_refl_b14,sur_refl_b15,QC_b8_15_1km
0,14245.0,2009.000000,2009_01_01,307.22,65.0,AR-Vir,2009.000000,NaN,2.0,0.0174,0.0300,0.0383,0.0666,0.0775,-0.0100,-0.010,-0.01,3.721396e+09
1,14246.0,2009.002740,2009_01_02,NaN,2.0,AR-Vir,2009.002740,NaN,2.0,0.4738,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09
2,14247.0,2009.005479,2009_01_03,NaN,2.0,AR-Vir,2009.005479,NaN,2.0,0.3668,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09
3,14248.0,2009.008219,2009_01_04,303.32,65.0,AR-Vir,2009.008219,306.48,0.0,0.2286,0.2401,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09
4,14249.0,2009.010959,2009_01_05,307.22,0.0,AR-Vir,2009.010959,309.16,0.0,0.0098,0.0150,0.0160,0.0288,0.0334,0.0271,0.027,-0.01,3.489661e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265172,14605.0,2009.986301,2009_12_27,NaN,2.0,ZM-Mon,2009.986301,NaN,2.0,0.2668,0.2599,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09
265173,14606.0,2009.989041,2009_12_28,NaN,2.0,ZM-Mon,2009.989041,NaN,2.0,0.1632,0.1658,0.1622,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722301e+09
265174,14607.0,2009.991781,2009_12_29,NaN,2.0,ZM-Mon,2009.991781,NaN,2.0,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09
265175,14608.0,2009.994521,2009_12_30,NaN,2.0,ZM-Mon,2009.994521,NaN,2.0,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.0100,-0.010,-0.01,3.722305e+09


In [20]:
%%R -i ddf
summary(ddf)   
# rows 265177 
# time 1991 - 2015

# !!!!!!!!!!!!!!! merging on year_dec loses about 1/3 of the data! 
# it is calculated differently and when merging there is a slight numeric difference. 
# merging on the date solves that

# length(unique(ddf$sites))  # 71

      date         year_dec_x        id            LST_Day_1km_terra
 Min.   : 7670   Min.   :1991   Length:265177      Min.   :234.2    
 1st Qu.:12464   1st Qu.:2004   Class :character   1st Qu.:284.3    
 Median :13919   Median :2008   Mode  :character   Median :293.6    
 Mean   :13696   Mean   :2007                      Mean   :292.5    
 3rd Qu.:15151   3rd Qu.:2011                      3rd Qu.:301.7    
 Max.   :16435   Max.   :2015                      Max.   :330.6    
                                                   NA's   :162614   
  QC_Day_terra       sites             year_dec_y   LST_Day_1km_aqua
 Min.   :  0.00   Length:265177      Min.   :1991   Min.   :233.9   
 1st Qu.:  2.00   Class :character   1st Qu.:2004   1st Qu.:286.1   
 Median :  2.00   Mode  :character   Median :2008   Median :295.0   
 Mean   : 12.91                      Mean   :2007   Mean   :294.9   
 3rd Qu.:  2.00                      3rd Qu.:2011   3rd Qu.:304.3   
 Max.   :145.00                   

# Merge reflectance data and land surface temperature

In [21]:
df_lst_and_refl = ddf.merge(refl_df, on=["id", "sites"])

# Export Data

In [22]:
df_lst_and_refl.to_csv('./data/df_lst_and_refl.csv', index=False)

Carry on with the Quality Control Notebook (02_QualityControl.ipynb)



# Overview

In [23]:
%%R -i df_lst_and_refl
summary(df_lst_and_refl)
# rows 241826                            was 265177 - about 91 %
# time 2001 - 2015                       ten years lost due to different timeframes, however at the beginning there is less data 
#                                        and no landsurface temperature (Aqua and Terra are launched in the early 2000s)
#                                        important to transform id with str.zfill(2) otherwise most of the data is lost

      date         year_dec_x        id            LST_Day_1km_terra
 Min.   :11011   Min.   :2000   Length:241826      Min.   :234.2    
 1st Qu.:12861   1st Qu.:2005   Class :character   1st Qu.:284.3    
 Median :14126   Median :2009   Mode  :character   Median :293.6    
 Mean   :14022   Mean   :2008                      Mean   :292.5    
 3rd Qu.:15254   3rd Qu.:2012                      3rd Qu.:301.7    
 Max.   :16434   Max.   :2015                      Max.   :330.6    
                                                   NA's   :139323   
  QC_Day_terra       sites             year_dec_y   LST_Day_1km_aqua
 Min.   :  0.00   Length:241826      Min.   :2000   Min.   :233.9   
 1st Qu.:  2.00   Class :character   1st Qu.:2005   1st Qu.:286.1   
 Median :  2.00   Mode  :character   Median :2009   Median :295.1   
 Mean   : 12.91                      Mean   :2008   Mean   :294.9   
 3rd Qu.:  2.00                      3rd Qu.:2012   3rd Qu.:304.4   
 Max.   :145.00                   

In [24]:
df_lst_and_refl = df_lst_and_refl[["sites", "year_dec_x", 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07',
                                 'sur_refl_b08', 'sur_refl_b09', 'sur_refl_b10', 
                                  'sur_refl_b11', 'sur_refl_b12', 'sur_refl_b13', 'sur_refl_b14']]

In [25]:
len(df_lst_and_refl.dropna())

235876

In [26]:
sites = pd.unique(df_lst_and_refl[['sites']].values.ravel('K'))

In [27]:
for s in sites:
    a= df_lst_and_refl[df_lst_and_refl["sites"]==s]
    print(f'{s} start: {int(a.iloc[0]["year_dec_x"])}, end: {int(a.iloc[len(a)-1]["year_dec_x"])} and has a size of {len(a)} (w/o NA: {len(a.dropna())})')

AR-Vir start: 2009, end: 2012 and has a size of 1459 (w/o NA: 1408)
AU-Ade start: 2007, end: 2009 and has a size of 1094 (w/o NA: 950)
AU-ASM start: 2010, end: 2013 and has a size of 1459 (w/o NA: 1355)
AU-DaP start: 2007, end: 2013 and has a size of 2553 (w/o NA: 2219)
AU-DaS start: 2008, end: 2014 and has a size of 2552 (w/o NA: 2342)
AU-Dry start: 2008, end: 2014 and has a size of 2552 (w/o NA: 2215)
AU-Fog start: 2006, end: 2008 and has a size of 1094 (w/o NA: 946)
AU-Gin start: 2011, end: 2014 and has a size of 1458 (w/o NA: 1448)
AU-How start: 2001, end: 2014 and has a size of 5074 (w/o NA: 4379)
AU-Stp start: 2008, end: 2014 and has a size of 2552 (w/o NA: 2218)
AU-Whr start: 2011, end: 2014 and has a size of 1458 (w/o NA: 1175)
AU-Wom start: 2010, end: 2012 and has a size of 1094 (w/o NA: 934)
BE-Bra start: 2000, end: 2014 and has a size of 5372 (w/o NA: 5328)
BE-Vie start: 2000, end: 2014 and has a size of 5372 (w/o NA: 5320)
CH-Fru start: 2005, end: 2014 and has a size of 364